In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pickle

with open('/content/train.pkl', 'rb') as f:
    train = pickle.load(f)

with open('/content/val.pkl', 'rb') as f:
    val = pickle.load(f)    

In [ ]:
train

,data,tMin,tMed,tMax,precipitacao,Produto,Vendas,Target,DIFF1_Vendas,DIFF2_Vendas,DIFF3_Vendas,DIFF4_Vendas,DIFF1_tMin,DIFF2_tMin,DIFF3_tMin,DIFF4_tMin,DIFF1_tMed,DIFF2_tMed,DIFF3_tMed,DIFF4_tMed,DIFF1_tMax,DIFF2_tMax,DIFF3_tMax,DIFF4_tMax,meses,semana,dias
0,2018-08-13,10.125,14.125,18.125,0,1,35.0,101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,33.0,13.0
1,2018-08-20,11.375,24.625,26.500,0,1,101.0,125,66.0,NaN,NaN,NaN,1.250,NaN,NaN,NaN,10.500,NaN,NaN,NaN,8.375,NaN,NaN,NaN,8.0,34.0,20.0
2,2018-08-27,13.000,17.250,21.250,0,1,125.0,82,24.0,90.0,NaN,NaN,1.625,2.875,NaN,NaN,-7.375,3.125,NaN,NaN,-5.250,3.125,NaN,NaN,8.0,35.0,27.0
3,2018-09-03,13.000,18.500,24.000,0,1,82.0,197,-43.0,-19.0,47.0,NaN,0.000,1.625,2.875,NaN,1.250,-6.125,4.375,NaN,2.750,-2.500,5.875,NaN,9.0,36.0,3.0
4,2018-09-10,10.750,15.375,20.125,0,1,197.0,113,115.0,72.0,96.0,162.0,-2.250,-2.250,-0.625,0.625,-3.125,-1.875,-9.250,1.250,-3.875,-1.125,-6.375,2.000,9.0,37.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2019-08-05,10.875,15.000,19.125,0,6,27.0,12,2.0,3.0,6.0,11.0,-2.875,0.000,1.250,0.125,-3.375,0.125,-0.125,0.875,-3.750,0.125,-1.375,1.750,8.0,31.0,5.0
276,2019-08-12,13.250,26.750,30.875,0,6,12.0,17,-15.0,-13.0,-12.0,-9.0,2.375,-0.500,2.375,3.625,11.750,8.375,11.875,11.625,11.750,8.000,11.875,10.375,8.0,32.0,12.0
277,2019-08-19,12.375,16.250,19.875,0,6,17.0,14,5.0,-10.0,-8.0,-7.0,-0.875,1.500,-1.375,1.500,-10.500,1.250,-2.125,1.375,-11.000,0.750,-3.000,0.875,8.0,33.0,19.0
278,2019-08-26,10.750,13.750,16.375,0,6,14.0,42,-3.0,2.0,-13.0,-11.0,-1.625,-2.500,-0.125,-3.000,-2.500,-13.000,-1.250,-4.625,-3.500,-14.500,-2.750,-6.500,8.0,34.0,26.0


In [ ]:
val

,data,tMin,tMed,tMax,precipitacao,Produto,Vendas,Target,DIFF1_Vendas,DIFF2_Vendas,DIFF3_Vendas,DIFF4_Vendas,DIFF1_tMin,DIFF2_tMin,DIFF3_tMin,DIFF4_tMin,DIFF1_tMed,DIFF2_tMed,DIFF3_tMed,DIFF4_tMed,DIFF1_tMax,DIFF2_tMax,DIFF3_tMax,DIFF4_tMax,meses,semana,dias
0,2019-09-09,13.375,17.250,20.625,0,1,90.0,137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,36.0,9.0
1,2019-09-16,16.875,22.500,28.000,0,1,137.0,82,47.0,NaN,NaN,NaN,3.500,NaN,NaN,NaN,5.250,NaN,NaN,NaN,7.375,NaN,NaN,NaN,9.0,37.0,16.0
2,2019-09-23,15.875,20.125,24.250,0,1,82.0,179,-55.0,-8.0,NaN,NaN,-1.000,2.500,NaN,NaN,-2.375,2.875,NaN,NaN,-3.750,3.625,NaN,NaN,9.0,38.0,23.0
3,2019-09-30,12.000,15.375,18.875,0,1,179.0,90,97.0,42.0,89.0,NaN,-3.875,-4.875,-1.375,NaN,-4.750,-7.125,-1.875,NaN,-5.375,-9.125,-1.750,NaN,9.0,39.0,30.0
4,2019-10-07,14.625,20.000,25.375,2,1,90.0,144,-89.0,8.0,-47.0,0.0,2.625,-1.250,-2.250,1.250,4.625,-0.125,-2.500,2.750,6.500,1.125,-2.625,4.750,10.0,40.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,2020-02-03,18.375,23.250,28.000,53,6,42.0,29,27.0,15.0,23.0,37.0,2.750,0.125,-1.125,0.375,2.875,1.750,-0.500,0.750,2.875,3.000,0.250,0.875,2.0,5.0,3.0
126,2020-02-10,18.625,21.875,25.125,22,6,29.0,36,-13.0,14.0,2.0,10.0,0.250,3.000,0.375,-0.875,-1.375,1.500,0.375,-1.875,-2.875,0.000,0.125,-2.625,2.0,6.0,10.0
127,2020-02-17,16.875,20.875,24.875,0,6,36.0,12,7.0,-6.0,21.0,9.0,-1.750,-1.500,1.250,-1.375,-1.000,-2.375,0.500,-0.625,-0.250,-3.125,-0.250,-0.125,2.0,7.0,17.0
128,2020-02-24,18.500,21.875,25.500,0,6,12.0,40,-24.0,-17.0,-30.0,-3.0,1.625,-0.125,0.125,2.875,1.000,0.000,-1.375,1.500,0.625,0.375,-2.500,0.375,2.0,8.0,24.0


In [ ]:
val.columns

Index(['data', 'tMin', 'tMed', 'tMax', 'precipitacao', 'Produto', 'Vendas',
       'Target', 'DIFF1_Vendas', 'DIFF2_Vendas', 'DIFF3_Vendas',
       'DIFF4_Vendas', 'DIFF1_tMin', 'DIFF2_tMin', 'DIFF3_tMin', 'DIFF4_tMin',
       'DIFF1_tMed', 'DIFF2_tMed', 'DIFF3_tMed', 'DIFF4_tMed', 'DIFF1_tMax',
       'DIFF2_tMax', 'DIFF3_tMax', 'DIFF4_tMax', 'meses', 'semana', 'dias'],
      dtype='object')

In [ ]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 3.0MB/s 


In [ ]:
%%time
from lightgbm import LGBMRegressor
from sklearn.metrics import median_absolute_error
c = 0
def run_rnn(params):
    estimators, num_leaves, lr = params

    features = ['tMin', 'tMed', 'tMax', 'Produto', 'Vendas',
       'DIFF1_Vendas', 'DIFF2_Vendas', 'DIFF3_Vendas','DIFF4_Vendas', 
       'DIFF1_tMin', 'DIFF2_tMin', 'DIFF3_tMin', 'DIFF4_tMin',
       'DIFF1_tMed', 'DIFF2_tMed', 'DIFF3_tMed', 'DIFF4_tMed', 'DIFF1_tMax',
       'DIFF2_tMax', 'DIFF3_tMax', 'DIFF4_tMax', 'semana']

    lgbm = LGBMRegressor(random_state=10, estimators=estimators, n_jobs=-1, num_leaves=num_leaves, learning_rate=lr)
    lgbm.fit(train[features].fillna(-1), train['Target'])

    preds = lgbm.predict(val[features].fillna(-1))
    e = median_absolute_error(val['Target'], preds)

    print("Ensemble Avg Score = {}".format(e))
    print("estimators = {}, num_leaves = {}, lr = {}".format(estimators, num_leaves, lr))
    print("The Best: estimators = {}, num_leaves = {}, lr = {}".format(estimators, num_leaves, lr))
    print('\n')

    return e

CPU times: user 30 µs, sys: 0 ns, total: 30 µs
Wall time: 36.2 µs


In [ ]:
from skopt import gp_minimize

space = [(10, 300),
         (2, 64),
         (1e-4, 1e-1, 'log-uniform')]

res = gp_minimize(run_rnn, space,random_state=0, verbose=1, n_calls=50)

Iteration No: 1 started. Evaluating function at random point.
Ensemble Avg Score = 5.695949608526011
estimators = 182, num_leaves = 54, lr = 0.03748321662847933
The Best: estimators = 182, num_leaves = 54, lr = 0.03748321662847933


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0490
Function value obtained: 5.6959
Current minimum: 5.6959
Iteration No: 2 started. Evaluating function at random point.
Ensemble Avg Score = 24.846675498334815
estimators = 256, num_leaves = 41, lr = 0.0014228041454890754
The Best: estimators = 256, num_leaves = 41, lr = 0.0014228041454890754


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0481
Function value obtained: 24.8467
Current minimum: 5.6959
Iteration No: 3 started. Evaluating function at random point.
Ensemble Avg Score = 26.21553948751209
estimators = 96, num_leaves = 6, lr = 0.0006576107192827547
The Best: estimators = 96, num_leaves = 6, lr = 0.0006576107192827547


Iteration No: 3 ended. Evaluation

In [ ]:
features = ['tMin', 'tMed', 'tMax', 'Produto', 'Vendas',
       'DIFF1_Vendas', 'DIFF2_Vendas', 'DIFF3_Vendas','DIFF4_Vendas', 
       'DIFF1_tMin', 'DIFF2_tMin', 'DIFF3_tMin', 'DIFF4_tMin',
       'DIFF1_tMed', 'DIFF2_tMed', 'DIFF3_tMed', 'DIFF4_tMed', 'DIFF1_tMax',
       'DIFF2_tMax', 'DIFF3_tMax', 'DIFF4_tMax', 'semana']

lgbm = LGBMRegressor(random_state=10, _estimators=500, n_jobs=-1, num_leaves=64, learning_rate=0.0360)
lgbm.fit(train[features].fillna(-1), train['Target'])

preds = lgbm.predict(val[features].fillna(-1))
median_absolute_error(val['Target'], preds)

5.973988038782859

In [ ]:
preds

array([120.5735975 , 108.55223648, 128.16257983, 110.13447991,
       135.40426072, 119.76350182, 148.08327882, 114.45580828,
       137.21199578, 124.66353619, 116.7967189 , 133.1661519 ,
       114.78505768, 131.76433123, 115.45777248, 113.75905364,
       142.70610015, 123.47262703, 120.74109848, 108.47532791,
       113.21566687, 129.59602174, 106.77070679, 119.96106046,
       102.81654411, 136.20835577,  63.88299512,  45.79284935,
        38.50573505,  51.0298186 ,  30.50645581,  24.96176412,
        39.62715655,  27.95652641,  33.82054086,  30.76567335,
        60.39341782,  40.35873458,  39.0534212 ,  22.94480527,
        31.29969693,  28.66492657,  41.85301041,  30.89973619,
        26.90230519,  53.54070573,  49.69829145,  29.88010409,
        33.48835267,  50.83865657,  21.98778943,  60.87323467,
         4.27360012,   7.97594598,   6.89956903,  12.69031445,
         4.6212663 ,   7.06156844,   6.87378829,   4.930492  ,
        13.07652107,   1.14172801,   8.73327115,   9.51